In [4]:
from dotenv import load_dotenv
load_dotenv() 

True

In [5]:
import os
from glob import glob 
from pprint import pprint 
import json 

## 1. RAG 검색기 (Retriever)

- RAG(Retrieval Augmented Generation)는 LLM이 답변을 생성할 때 외부 지식 소스에서 관련 정보를 검색하여 참고하도록 하는 방식임. 
- 이 '검색' 단계를 수행하는 것이 바로 검색기(Retriever)임.
- 벡터 저장소는 이 검색기를 구현하는 강력한 도구임.

### 1.1 Semantic Search (의미론적 검색)

- 단어의 단순 매칭이 아닌, 문맥적 의미의 유사성을 기반으로 문서를 검색하는 방식임.
- 벡터 저장소와 임베딩 모델을 활용하여 구현됨.
- **장점:**
  - 동의어, 유의어, 문맥적 유사성 파악 가능.
  - 키워드가 정확히 일치하지 않아도 관련 문서 검색 가능.
- **단점:**
  - 임베딩 모델의 성능에 크게 의존함.
  - 특정 키워드가 매우 중요한 경우, 오히려 키워드 검색보다 성능이 낮을 수 있음.
  - 계산 비용이 키워드 검색보다 높을 수 있음.

`(1) 벡터 저장소 초기화 (의미론적 검색용)`

- 실제 텍스트 파일들을 로드하고, 문장 단위로 분할한 후, Chroma DB에 저장함.
- `TextLoader`: 텍스트 파일을 로드함.
- `CharacterTextSplitter.from_huggingface_tokenizer`: 임베딩 모델이 사용하는 토크나이저를 기준으로 텍스트를 분할하여, 토큰 수 제한에 맞추면서 의미를 최대한 보존하려 함.
  - `separator=r"[.!?]\s+"`: 마침표, 느낌표, 물음표 뒤에 공백이 오는 경우를 문장 구분자로 사용 (정규식).
  - `chunk_size`, `chunk_overlap`: 분할된 청크의 최대 크기와 청크 간 중복되는 토큰 수를 지정. 중복은 문맥 유실을 줄이는 데 도움됨.
- Chroma DB 생성 시 `collection_metadata = {'hnsw:space': 'cosine'}`
  - 벡터 간 유사도 계산 시 코사인 유사도(cosine distance)를 사용하도록 명시.
  - 코사인 유사도는 벡터의 크기보다 방향에 중점을 두므로 텍스트 유사도 측정에 자주 사용됨.

In [6]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from transformers import AutoTokenizer

In [7]:
# 데이터 로드 함수
def load_text_files(txt_files):
    data = []
    for text_file in txt_files:
        print(f"로딩 중: {text_file}")
        # encoding='utf-8' 명시하여 한글 파일 로드 문제 방지
        loader = TextLoader(text_file, encoding='utf-8') 
        data.extend(loader.load())
    return data

In [8]:
korean_txt_files = glob(os.path.join('data', '*_KR.txt'))
if not korean_txt_files:
    print("'data' 폴더에 '*_KR.txt' 파일이 없습니다. 예시 데이터를 생성하거나 경로를 확인하세요.")
    korean_data = [] # 빈 리스트로 초기화
else:
    korean_data = load_text_files(korean_txt_files)

print(f"\n로드된 전체 문서 수: {len(korean_data)}")
if korean_data:
    print(f"첫 번째 문서 내용 일부: {korean_data[0].page_content[:100]}...")


로딩 중: data\Rivian_KR.txt
로딩 중: data\Tesla_KR.txt

로드된 전체 문서 수: 2
첫 번째 문서 내용 일부: 2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당...


In [10]:
# Hugging Face 임베딩 모델(BAAI/bge-m3)이 사용하는 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

# 문장 구분자(정규식)를 사용하여 텍스트 분할기 생성
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,          # 토큰 수 계산 기준
    separator=r"[.!?]\s+",     # 문장 구분자: 마침표, 느낌표, 물음표 뒤 공백
    chunk_size=100,             # 청크 최대 토큰 수 (토크나이저 기준)
    chunk_overlap=20,            # 청크 간 중복 토큰 수
    is_separator_regex=True,    # separator가 정규식임을 명시
    keep_separator=True,        # 구분자 유지 여부
)

korean_docs = []
if korean_data: 
    korean_docs = text_splitter.split_documents(korean_data)

print(f"\n분할된 한국어 문서 수: {len(korean_docs)}")
if korean_docs:
    print(f"첫 번째 분할 문서 내용: {korean_docs[0].page_content}")
    print(f"첫 번째 분할 문서 메타데이터: {korean_docs[0].metadata}")


분할된 한국어 문서 수: 8
첫 번째 분할 문서 내용: 2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다
첫 번째 분할 문서 메타데이터: {'source': 'data\\Rivian_KR.txt'}


In [11]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# Hugging Face 임베딩 모델 재사용
embeddings_huggingface = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# Chroma 벡터 저장소 생성 (코사인 유사도 사용 명시)
if korean_docs: # 분할된 문서가 있을 경우에만 DB 생성
    chroma_db_semantic = Chroma.from_documents(
        documents=korean_docs,
        embedding=embeddings_huggingface,
        collection_name="db_korean_cosine", # 새 컬렉션 이름
        persist_directory="./chroma_db_semantic", # 새 저장 경로
        collection_metadata={'hnsw:space': 'cosine'}, # 유사도 계산 방식: 코사인
    )
    print("의미론적 검색용 Chroma DB 생성 완료.")
else:
    print("분할된 문서가 없어 의미론적 검색용 Chroma DB를 생성하지 않았습니다.")
    chroma_db_semantic = None # DB가 생성되지 않았음을 명시

의미론적 검색용 Chroma DB 생성 완료.


`(2) Top K 검색`

- `vector_store.as_retriever(search_kwargs={"k": N})` 형태로 검색기를 생성함.
- 가장 기본적인 검색 방식으로, 쿼리와 유사도가 높은 상위 N개의 문서를 반환함.
- **장점:** 사용이 간편하고 직관적임.
- **단점:** 반환되는 문서들이 서로 유사하여 다양성이 부족할 수 있음. 최적의 `k`값 설정이 필요.

In [12]:
if chroma_db_semantic:
    chroma_k_retriever = chroma_db_semantic.as_retriever(
        search_kwargs={"k": 2}, # 상위 2개 문서 검색
    )

    query = "리비안은 언제 사업을 시작했나요?"
    retrieved_docs_k = chroma_k_retriever.invoke(query)

    print(f"쿼리: {query}")
    print("Top K 검색 결과:")
    for doc in retrieved_docs_k:
        print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
else:
    print("Chroma DB (semantic)가 초기화되지 않아 Top K 검색을 수행할 수 없습니다.")

쿼리: 리비안은 언제 사업을 시작했나요?
Top K 검색 결과:
- 2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다 [출처: data\Rivian_KR.txt]
- 2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다 [출처: data\Rivian_KR.txt]


`(3) 임계값 지정 검색 (Similarity Score Threshold)`

- `search_type='similarity_score_threshold'` 와 `search_kwargs={'score_threshold': T, 'k': N}`을 사용.
- 코사인 유사도(0~1, 높을수록 유사) 기준으로, 설정한 임계값(T) 이상인 문서들 중에서 상위 N개를 반환함.
- **장점:** 관련성이 낮은 문서를 효과적으로 필터링 가능. 검색 결과의 질을 높일 수 있음.
- **단점:** 적절한 임계값(threshold) 설정이 중요함. 너무 높으면 필요한 문서를 놓칠 수 있고, 너무 낮으면 의미 없는 문서가 포함될 수 있음.

In [14]:
from langchain_community.utils.math import cosine_similarity # 코사인 유사도 계산 함수

if chroma_db_semantic:
    chroma_threshold_retriever = chroma_db_semantic.as_retriever(
        search_type='similarity_score_threshold', # 유사도 점수 임계값 기반 검색
        search_kwargs={'score_threshold': 0.5, 'k': 2}, # 코사인 유사도 0.5 이상, 상위 2개
    )

    query = "리비안은 언제 사업을 시작했나요?"
    retrieved_docs_threshold = chroma_threshold_retriever.invoke(query)

    print(f"쿼리: {query}")
    print("임계값 지정 검색 결과:")
    for doc in retrieved_docs_threshold:
        # 검색 결과 문서와 쿼리 간의 코사인 유사도 직접 계산 (확인용)
        query_embedding = embeddings_huggingface.embed_query(query)
        doc_embedding = embeddings_huggingface.embed_query(doc.page_content)
        score = cosine_similarity([query_embedding], [doc_embedding])[0][0]
        print(f"- {doc.page_content} [유사도: {score:.4f}, 출처: {doc.metadata['source']}]")
else:
    print("Chroma DB (semantic)가 초기화되지 않아 임계값 지정 검색을 수행할 수 없습니다.")

쿼리: 리비안은 언제 사업을 시작했나요?
임계값 지정 검색 결과:
- 2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다 [유사도: 0.6882, 출처: data\Rivian_KR.txt]
- 2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다 [유사도: 0.6882, 출처: data\Rivian_KR.txt]


`(4) MMR (Maximal Marginal Relevance) 검색`

- `search_type='mmr'` 와 `search_kwargs={'k': N, 'fetch_k': M, 'lambda_mult': L}`을 사용.
- 검색 결과의 **정확성(유사도)**과 **다양성**을 모두 고려하는 방식임.
  - 먼저 `fetch_k` 만큼의 후보 문서를 가져온 후, MMR 알고리즘을 사용하여 최종 `k`개의 문서를 선택함.
  - `lambda_mult` (0~1 사이 값): 0에 가까울수록 다양성 중시, 1에 가까울수록 유사도(정확성) 중시. 기본값은 0.5.
- **장점:** 유사하면서도 다양한 관점의 문서를 함께 제공하여 사용자에게 더 풍부한 정보 제공 가능.
- **단점:** `fetch_k`, `lambda_mult` 등 추가 파라미터 튜닝이 필요할 수 있음. 일반 유사도 검색보다 계산량이 많을 수 있음.

In [15]:
if chroma_db_semantic:
    chroma_mmr_retriever = chroma_db_semantic.as_retriever(
        search_type='mmr', # MMR 검색 방식
        search_kwargs={
            'k': 3,             # 최종 반환할 문서 수
            'fetch_k': 8,      # MMR 계산을 위해 초기에 가져올 문서 수 (k보다 커야 함)
            'lambda_mult': 0.5, # 다양성 vs 유사도 가중치 (0: 최대 다양성, 1: 최소 다양성)
        },
    )

    query = "리비안은 언제 사업을 시작했나요?"
    retrieved_docs_mmr = chroma_mmr_retriever.invoke(query)

    print(f"쿼리: {query}")
    print("MMR 검색 결과:")
    for doc in retrieved_docs_mmr:
        query_embedding = embeddings_huggingface.embed_query(query)
        doc_embedding = embeddings_huggingface.embed_query(doc.page_content)
        score = cosine_similarity([query_embedding], [doc_embedding])[0][0]
        print(f"- {doc.page_content} [유사도: {score:.4f}, 출처: {doc.metadata['source']}]")
else:
    print("Chroma DB (semantic)가 초기화되지 않아 MMR 검색을 수행할 수 없습니다.")

쿼리: 리비안은 언제 사업을 시작했나요?
MMR 검색 결과:
- 2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다 [유사도: 0.6882, 출처: data\Rivian_KR.txt]
- .

리비안의 초기 프로젝트는 피터 스티븐스가 디자인한 2+2 시트 배열의 미드십 엔진 하이브리드 쿠페 스포츠카 R1(원래 이름은 아베라(Avera))이었습니다. 이 차량은 모듈식 캡슐 구조와 쉽게 교체 가능한 차체 패널을 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산을 시작할 것으로 예상되었습니다 [유사도: 0.5927, 출처: data\Rivian_KR.txt]
- .

2021년 10월, 리비안은 첫 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다. [유사도: 0.5650, 출처: data\Rivian_KR.txt]


`(5) 메타데이터 필터링 검색`

- `search_kwargs={'filter': {'metadata_key': 'metadata_value'}, 'k': N}` 형태로 사용.
- 문서에 저장된 메타데이터를 기준으로 검색 대상을 한정한 후, 그 안에서 유사도 높은 N개 문서를 찾음.
- **장점:** 특정 출처, 카테고리 등 원하는 조건의 문서 내에서만 검색 가능하여 효율적이고 정확한 검색 가능.
- **단점:** 메타데이터가 잘 정의되고 일관성 있게 관리되어야 효과적임.

In [16]:
if chroma_db_semantic:
    target_source_file = './data/Rivian_KR.txt' 
    if not os.path.exists(target_source_file):
        print(f"경고: '{target_source_file}' 파일이 없어 메타데이터 필터링이 제대로 동작하지 않을 수 있습니다.")

    chroma_metadata_retriever = chroma_db_semantic.as_retriever(
        search_kwargs={
            'filter': {'source': target_source_file}, # 'source' 메타데이터가 특정 파일인 문서만 대상
            'k': 1, 
        }
    )

    query = "리비안의 주요 모델은 무엇인가요?"
    retrieved_docs_metadata = chroma_metadata_retriever.invoke(query)

    print(f"쿼리: {query}")
    print(f"메타데이터 필터링 검색 결과 (출처: {target_source_file}):")
    for doc in retrieved_docs_metadata:
        print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
else:
    print("Chroma DB (semantic)가 초기화되지 않아 메타데이터 필터링 검색을 수행할 수 없습니다.")

쿼리: 리비안의 주요 모델은 무엇인가요?
메타데이터 필터링 검색 결과 (출처: ./data/Rivian_KR.txt):


### 1.2 Keyword Search (키워드 검색)

- TF-IDF, BM25 등 전통적인 정보 검색 알고리즘을 사용하여, 쿼리에 포함된 키워드와 문서 내 키워드의 일치도를 기반으로 검색함.
- `rank_bm25` 패키지 설치 필요
- **장점:**
  - 속도가 빠르고 계산 비용이 낮음.
  - 특정 키워드가 중요한 검색에서 높은 정확도를 보일 수 있음.
  - 구현이 비교적 간단함.
- **단점:**
  - 동의어, 유의어 등 의미론적 유사성을 파악하지 못함 (예: '자동차'와 '탈것'을 다르게 인식).
  - 단어의 순서나 문맥을 고려하지 않음.
  - 한국어의 경우 형태소 분석(토큰화) 품질이 검색 결과에 큰 영향을 미침.

`(1) BM25 검색기 생성`

- BM25 (Best Matching 25)는 TF-IDF를 개선한 알고리즘으로, 문서 길이까지 고려하여 점수를 매김.
- `BM25Retriever.from_documents(docs)`를 사용하여 기존 `Document` 객체들로부터 BM25 검색기를 쉽게 생성할 수 있음.
- 기본적으로 공백 기준으로 단어를 분리(토큰화)하여 사용함.

In [ ]:
if chroma_db_semantic:
    bm25_input_docs = korean_docs # 이미 Document 객체 리스트임
    print(f"BM25 검색기 생성을 위한 문서 수: {len(bm25_input_docs)}")

    from langchain_community.retrievers import BM25Retriever
    
    bm25_retriever_simple = BM25Retriever.from_documents(bm25_input_docs)
    print("기본 BM25 검색기 생성 완료.")

    query = "리비안은 언제 사업을 시작했나요?"
    retrieved_docs_bm25_simple = bm25_retriever_simple.invoke(query)

    print(f"\n쿼리: {query}")
    print("기본 BM25 검색 결과:")
    for doc in retrieved_docs_bm25_simple:
        print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
else:
    print("korean_docs가 없어 BM25 검색기를 생성할 수 없습니다.")
    bm25_retriever_simple = None

BM25 검색기 생성을 위한 문서 수: 8
기본 BM25 검색기 생성 완료.

쿼리: 리비안은 언제 사업을 시작했나요?
기본 BM25 검색 결과:
- .

2021년 10월, 리비안은 첫 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다. [출처: data\Rivian_KR.txt]
- . 리비안은 디젤 하이브리드, 브라질 원메이크 시리즈를 위한 R1 GT라는 이름의 레이싱 버전, 4도어 세단, 크로스오버 등 다양한 버전을 검토했습니다. 2011년에 프로토타입 해치백이 공개되었지만, R1과의 연관성은 아직 불분명합니다 [출처: data\Rivian_KR.txt]
- 2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다 [출처: data\Rivian_KR.txt]
- . SMT 패키징(SMT Packaging)에 따르면, 테슬라의 2023년 판매량은 글로벌 전기 자동차 시장의 약 12.9%를 차지했습니다. [출처: data\Tesla_KR.txt]


In [ ]:
if bm25_retriever_simple:
    # BM25 점수 확인 (내부 vectorizer 사용)
    query_bm25_score_check = "리비안은 언제 사업을 시작했나요?"
    tokenized_query_bm25 = query_bm25_score_check.split() # 기본 공백 토큰화
    print(f"토큰화된 쿼리 (공백 기준): {tokenized_query_bm25}")

    print("BM25 점수는 invoke 결과의 순서로 유추할 수 있음 (높은 순으로 정렬됨).")
else:
    print("BM25 검색기가 없어 점수를 확인할 수 없습니다.")

토큰화된 쿼리 (공백 기준): ['리비안은', '언제', '사업을', '시작했나요?']
BM25 점수는 invoke 결과의 순서로 유추할 수 있음 (높은 순으로 정렬됨).


In [19]:
# 의미는 같지만 다른 단어를 사용한 쿼리 테스트
if bm25_retriever_simple:
    query_synonym = "리비안이 설립된 연도는?" # '사업 시작' -> '설립 연도'
    retrieved_docs_bm25_synonym = bm25_retriever_simple.invoke(query_synonym)

    print(f"쿼리: {query_synonym}")
    print("유의어 사용 시 기본 BM25 검색 결과:")
    for doc in retrieved_docs_bm25_synonym:
        print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("\n-> 기본 BM25는 '설립'과 '시작'의 의미적 유사성을 모르므로, 키워드가 다르면 결과가 달라짐.")
else:
    print("BM25 검색기가 없어 유의어 테스트를 수행할 수 없습니다.")

쿼리: 리비안이 설립된 연도는?
유의어 사용 시 기본 BM25 검색 결과:
- . SMT 패키징(SMT Packaging)에 따르면, 테슬라의 2023년 판매량은 글로벌 전기 자동차 시장의 약 12.9%를 차지했습니다. [출처: data\Tesla_KR.txt]
- . 테슬라는 2010년 6월 나스닥에 상장되었습니다.

2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년 대비 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 테슬라의 누적 글로벌 판매량은 4,962,975대를 넘어섰습니다 [출처: data\Tesla_KR.txt]
- . 페이팔(PayPal)과 짚투(Zip2)의 공동 창립자인 머스크는 최대 주주이자 회장이 되어 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 저명한 물리학자이자 전기 공학자인 니콜라 테슬라(Nikola Tesla)의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다 [출처: data\Tesla_KR.txt]
- 텍사스주 오스틴에 본사를 둔 테슬라(Tesla, Inc.)는 미국의 대표적인 전기 자동차 제조업체입니다. 2003년 마틴 에버하드(Martin Eberhard, CEO)와 마크 타페닝(Marc Tarpenning, CFO)이 설립한 테슬라는 2004년 일론 머스크(Elon Musk)의 적극적인 참여를 받았습니다 [출처: data\Tesla_KR.txt]

-> 기본 BM25는 '설립'과 '시작'의 의미적 유사성을 모르므로, 키워드가 다르면 결과가 달라짐.


`(2) Kiwi 한국어 토크나이저 활용`

- 한국어는 조사가 발달하고 어미 변화가 다양하여, 단순 공백 기반 토큰화로는 성능이 낮음.
- `kiwipiepy` 같은 한국어 형태소 분석기를 사용하여 토큰화하면 BM25 검색 성능을 크게 향상시킬 수 있음.
- `BM25Retriever`의 `preprocess_func` 인자에 사용자 정의 토큰화 함수를 전달함.
- 사용자 사전(`add_user_word`)을 통해 '리비안', '테슬라' 같은 고유명사를 올바르게 인식하도록 할 수 있음.
- **장점:** 한국어 특성에 맞는 토큰화로 검색 정확도 향상, 사용자 사전으로 특정 단어 처리 개선.
- **단점:** 형태소 분석기 모델 로딩 시간 및 처리 시간 추가, 토크나이저 설정 및 관리 필요.

In [47]:
# !pip install kiwipiepy # 설치 필요 시 주석 해제
from kiwipiepy import Kiwi # Kiwi 형태소 분석기

# Kiwi 토크나이저를 사용한 전처리 함수
def kiwi_bm25_process_func(text):
    kiwi = Kiwi() # 함수 호출 시마다 Kiwi 객체 생성 (성능 고려 시 외부에서 생성 후 전달 권장)
    # 사용자 사전 추가 (고유명사 등)
    kiwi.add_user_word('리비안', 'NNP') # NNP: 고유명사 품사 태그
    kiwi.add_user_word('테슬라', 'NNP')
    # 텍스트를 토큰화하여 형태소(form) 리스트 반환
    return [token.form for token in kiwi.tokenize(text)]

if korean_docs: # 이전 단계의 korean_docs 사용
    bm25_retriever_kiwi = BM25Retriever.from_documents(
        documents=korean_docs,
        preprocess_func=kiwi_bm25_process_func, # Kiwi 토큰화 함수 사용
    )
    print("Kiwi 토크나이저 적용 BM25 검색기 생성 완료.")

    # 이전과 동일한 유의어 쿼리로 검색
    query_synonym_kiwi = "리비안이 설립된 연도는?"
    retrieved_docs_bm25_kiwi = bm25_retriever_kiwi.invoke(query_synonym_kiwi)

    print(f"\n쿼리: {query_synonym_kiwi}")
    print("Kiwi 토크나이저 적용 BM25 검색 결과:")
    for doc in retrieved_docs_bm25_kiwi:
        print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("\n-> Kiwi를 사용하면 '설립', '연도' 등의 키워드가 더 잘 분리되어 매칭될 가능성 높아짐.")
    
    # Kiwi 토크나이저로 쿼리가 어떻게 토큰화되는지 확인
    kiwi_instance = Kiwi()
    kiwi_instance.add_user_word('리비안', 'NNP')
    tokenized_query_kiwi_check = [t.form for t in kiwi_instance.tokenize(query_synonym_kiwi)]
    print(f"Kiwi로 토큰화된 쿼리: {tokenized_query_kiwi_check}")
else:
    print("korean_docs가 없어 Kiwi 적용 BM25 검색기를 생성할 수 없습니다.")
    bm25_retriever_kiwi = None

Kiwi 토크나이저 적용 BM25 검색기 생성 완료.

쿼리: 리비안이 설립된 연도는?
Kiwi 토크나이저 적용 BM25 검색 결과:
- 2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다 [출처: data\Rivian_KR.txt]
- 텍사스주 오스틴에 본사를 둔 테슬라(Tesla, Inc.)는 미국의 대표적인 전기 자동차 제조업체입니다. 2003년 마틴 에버하드(Martin Eberhard, CEO)와 마크 타페닝(Marc Tarpenning, CFO)이 설립한 테슬라는 2004년 일론 머스크(Elon Musk)의 적극적인 참여를 받았습니다 [출처: data\Tesla_KR.txt]
- . 페이팔(PayPal)과 짚투(Zip2)의 공동 창립자인 머스크는 최대 주주이자 회장이 되어 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 저명한 물리학자이자 전기 공학자인 니콜라 테슬라(Nikola Tesla)의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다 [출처: data\Tesla_KR.txt]
- . 리비안은 디젤 하이브리드, 브라질 원메이크 시리즈를 위한 R1 GT라는 이름의 레이싱 버전, 4도어 세단, 크로스오버 등 다양한 버전을 검토했습니다. 2011년에 프로토타입 해치백이 공개되었지만, R1과의 연관성은 아직 불분명합니다 [출처: data\Rivian_KR.txt]

-> Kiwi를 사용하면 '설립', '연도' 등의 키워드가 더 잘 분리되어 매칭될 가능성 높아짐.
Kiwi로 토큰화된 쿼리: ['리비안', '이', '설립', '되', 'ᆫ', '연도', '는', '?']


### 3.3 Hybrid Search (하이브리드 검색)

- 의미론적 검색(Semantic Search)과 키워드 검색(Keyword Search)의 장점을 결합하는 방식임.
- `EnsembleRetriever`를 사용하여 여러 검색기(예: Chroma 검색기, BM25 검색기)를 하나로 묶고, 각 검색 결과에 가중치(`weights`)를 부여하여 최종 순위를 결정함.
- **장점:**
  - 의미론적 유사성과 키워드 정확성을 모두 고려하여 검색 성능 향상 기대.
  - 한 가지 검색 방식의 단점을 다른 방식이 보완해 줄 수 있음.
- **단점:**
  - 여러 검색기를 사용하므로 계산 비용 증가.
  - 각 검색기의 성능 및 가중치(`weights`) 튜닝이 중요하며, 최적값 찾기가 어려울 수 있음.
  - 구현 및 관리가 더 복잡해짐.

In [48]:
from langchain.retrievers import EnsembleRetriever

# 이전 단계에서 생성한 검색기들을 사용
# chroma_threshold_retriever (의미론적 검색기, 임계값 기반)
# bm25_retriever_kiwi (키워드 검색기, Kiwi 토크나이저 사용)

if chroma_threshold_retriever and bm25_retriever_kiwi: # 두 검색기가 모두 준비되었을 때만 실행
    ensemble_retriever = EnsembleRetriever(
        retrievers=[chroma_threshold_retriever, bm25_retriever_kiwi],
        weights=[0.5, 0.5]  # 각 검색기 결과에 대한 가중치 (합이 1이 아니어도 됨, 상대적 비율)
    )
    print("Ensemble Retriever (하이브리드 검색기) 생성 완료.")

    query_hybrid = "리비안이 설립된 연도는?"
    retrieved_docs_hybrid = ensemble_retriever.invoke(query_hybrid)

    print(f"\n쿼리: {query_hybrid}")
    print("하이브리드 검색 결과:")
    # EnsembleRetriever는 중복된 문서를 자동으로 제거해줌
    for doc in retrieved_docs_hybrid:
        print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
else:
    print("의미론적 검색기 또는 BM25 검색기가 준비되지 않아 하이브리드 검색기를 생성할 수 없습니다.")

Ensemble Retriever (하이브리드 검색기) 생성 완료.

쿼리: 리비안이 설립된 연도는?
하이브리드 검색 결과:
- 2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다 [출처: data\Rivian_KR.txt]
- 텍사스주 오스틴에 본사를 둔 테슬라(Tesla, Inc.)는 미국의 대표적인 전기 자동차 제조업체입니다. 2003년 마틴 에버하드(Martin Eberhard, CEO)와 마크 타페닝(Marc Tarpenning, CFO)이 설립한 테슬라는 2004년 일론 머스크(Elon Musk)의 적극적인 참여를 받았습니다 [출처: data\Tesla_KR.txt]
- . 페이팔(PayPal)과 짚투(Zip2)의 공동 창립자인 머스크는 최대 주주이자 회장이 되어 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 저명한 물리학자이자 전기 공학자인 니콜라 테슬라(Nikola Tesla)의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다 [출처: data\Tesla_KR.txt]
- . 리비안은 디젤 하이브리드, 브라질 원메이크 시리즈를 위한 R1 GT라는 이름의 레이싱 버전, 4도어 세단, 크로스오버 등 다양한 버전을 검토했습니다. 2011년에 프로토타입 해치백이 공개되었지만, R1과의 연관성은 아직 불분명합니다 [출처: data\Rivian_KR.txt]
